In [1]:
junction_id = {'KLONGTEI': 'cluster_1892287670_272491964_272492178',
      'RAMA4': 'cluster_272488163_282390730_66263210_66263222',
      'NARANONG': 'cluster_272488164_272492179_3457051443_61907354',
      'SUNLAKAKHON': 'gneJ83',
      'KASEMRAT': 'cluster_272448137_272555800_272555808_7660045934_7710268409',
      'ATTHAKAWI_RAMA4' : '270329335'}
junction_name = list(junction_id.keys())

In [2]:
# import gym
# from gym import error, spaces
import gym
import lxml.etree as ET
import csv
import json
import os, sys
sys.path.append(os.path.join('/home/ring/sumo-svn/', 'tools'))
import traci
import traci.constants as tc
import numpy as np
from sumolib import checkBinary
import datetime
import matplotlib
import matplotlib.pyplot as plt
import torch
import copy
import random
import pandas as pd
from multiprocessing.pool import ThreadPool
from itertools import zip_longest

In [3]:
namelane_csv = pd.read_csv('namelane_KASEMRAT.csv')
namelane_df = pd.DataFrame(namelane_csv, columns = ['name' , 'id'])
NAME = namelane_df.set_index('name')
ID = namelane_df.set_index('id')
if NAME.loc['KASEMRAT_EB_0_0_XSXX','id'] == '459551209#3_0':
    print('ok')

ok


In [4]:
NAME.loc[:,'id']

name
KASEMRAT_EB_0_0_XSXX                      459551209#3_0
KASEMRAT_EB_0_1_XSXX                      459551209#3_1
KASEMRAT_EB_0_2_XSXX                      459551209#3_2
KASEMRAT_EB_0_3_XSRT                      459551209#3_3
KASEMRAT_EB_1_0_LSXX                      459551209#0_0
                                              ...      
MASUKGRIDLOCK_ATTHAKAWI_SB_2_1_LXXX        27702347#6_1
MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT        27702347#4_0
MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX        27702347#6_0
MASUKGRIDLOCK_SUKHUMVUT22_SB_0_0_XSXX    -453669106#1_0
MASUKGRIDLOCK_SUKHUMVUT24_SB_0_0_XSXX     328942767#2_0
Name: id, Length: 425, dtype: object

In [5]:
ID.loc['820373198#0_3', 'name']

'RAMA4_WB_2_3_XSXX'

In [6]:
namedetector_csv = pd.read_csv('namedetector_KASEMRAT_flow.csv')
namedetector_df = pd.DataFrame(namedetector_csv, columns = ['name' , 'id'])
NAME_D = namedetector_df.set_index('name')
ID_D = namedetector_df.set_index('id')
listdetector = open("namedetector_KASEMRAT_flow.txt", "r")
detector = {}
for l in listdetector:
    l = l.strip().split(' ')
    if len(l)> 1:
        d = []
        for detec in l[2:]:
            if type(NAME_D.loc[detec,'id']) == str:
                d.append(NAME_D.loc[detec,'id'])
            else : d.append(NAME_D.loc[detec,'id'][0])
    if str(l[0])!= '':
        detector[str(l[0])] = d
list_detector= list(detector.keys())

In [7]:
NAME_D.head()

,id
name,
RAMA4_EB_1_4_XSXX,D459492917#0_4
KLONGTEI_EB_0_0_XSXX,D825786400_0
KLONGTEI_EB_0_1_XSXX,D825786400_1
KLONGTEI_EB_0_2_XSXX,D825786400_2
KLONGTEI_EB_0_3_XSXX,D825786400_3


In [8]:
# type(NAME_D.loc['KLONGTEI_EB_0_0_XSXX','id'])
# for i in range(len(list(detector.keys()))):
#     print(i , list(detector.keys())[i])

In [9]:
def get_hot_encoding_current_phase():
    number_phase = [4,9,7,5,4,3]
    current_phase = [traci.trafficlight.getPhase(junction_id[key]) for key in junction_id.keys()]
#     current_phase = [0,2,1,1,1,1]
    hot_encoding_current_phase = np.array([])
    for i in range(len(current_phase)):
        binary_phase = np.zeros(number_phase[i])
        binary_phase[current_phase[i]] = 1
        hot_encoding_current_phase = np.concatenate((hot_encoding_current_phase, binary_phase), axis=None)
    return current_phase, hot_encoding_current_phase


In [10]:
# ID.loc['']#, '820373198#0', '820373196#0', '482209831#0']

In [11]:
def get_state_attention(current_phase, hot_encoding_current_phase):
#     current_phase = [KLONGTEI phase, RAMA4 phase,NARANONG phase, SUNLAKAKHON phase, KASEMRAT phase, ATTHAKAWI_RAMA4 phase]
    MAP_RAMA4 = {0: ['RAMA4_EB_FPX_TP1', 'KASEMRAT_EB_FPX_TP2_RAMA4', 'NARANONG_SW_FPX_TP1'],
                 1: ['RAMA4_EB_FP1_TP3', 'RAMA4_NB_FPX_TP5'],
                 2: ['RAMA4_WB_FP2_TP4', 'RAMA4_EB_FPX_TP1', 'RAMA4_NB_FPX_TP5'],
                3: ['RAMA4_EB_FP3_TP2', 'RAMA4_WB_FP3_TP2', 'RAMA4_EB_FPX_TP1', 'RAMA4_NB_FPX_TP5' ],
                4: ['RAMA4_SB_FP4_TP5', 'RAMA4_EB_FPX_TP1', 'RAMA4_NB_FPX_TP5' ],
                5: ['RAMA4_NB_FP5_TP1', 'RAMA4_EB_FPX_TP1'],
                6: ['KASEMRAT_EB_FPX_TP2_RAMA4'],
                7: ['KASEMRAT_EB_FPX_TP2_RAMA4'],
                8: ['KASEMRAT_EB_FPX_TP2_RAMA4']}
    MAP_KLONGTEI = {0: ['NARANONG_SB_FP5_TP6', 'RAMA4_EB_FP1_TP3'],
                    1: ['RAMA4_EB_FP3_TP2', 'RAMA4_EB_FP1_TP3'],
                    2: ['KLONGTEI_NB_FP3_TPX'],
                   3: ['RAMA4_EB_FP3_TP2',  'RAMA4_EB_FP1_TP3'] }
    MAP_NARANONG = {0: ['RAMA4_NB_FPX_TP5'],
                1: ['NARANONG_SB_FP1_TP2', 'NARANONG_EB_FP1_TP2', 'NARANONG_WB_FPX_TP2', 'NARANONG_WB_FPX_TP2'],
                2: ['NARANONG_WB_FP2_TP3', 'NARANONG_WB_FP2_TP3_FLOW', 'NARANONG_SW_FPX_TP1', 'NARANONG_WB_FPX_TP2'],
                3: ['NARANONG_EB_FP3_TP4', 'NARANONG_SW_FPX_TP1', 'NARANONG_WB_FPX_TP2'],
                4: ['NARANONG_WB_FP4_TP5', 'NARANONG_SW_FPX_TP1', 'NARANONG_WB_FPX_TP2'],
                5: ['NARANONG_SB_FP5_TP6', 'NARANONG_SW_FPX_TP1', 'NARANONG_WB_FPX_TP2'], 
                6: ['NARANONG_NB_FP6_TP1', 'NARANONG_SW_FPX_TP1', 'NARANONG_WB_FPX_TP2'] }
    MAP_SUNLAKAKHON = {0: ['NARANONG_EB_FP3_TP4'],
                    1: ['SUNLAKAKHON_SB_FP1_TPX'],
                    2: ['SUNLAKAKHON_NB_FP2_TP3', 'SUNLAKAKHON_SB_FPX_TP1'],
                    3:['SUNLAKAKHON_EB_FP3_TP4', 'SUNLAKAKHON_WB_FP3_TP4', 'SUNLAKAKHON_SB_FPX_TP1'],
                    4:['SUNLAKAKHON_SB_FP4_TP1', 'SUNLAKAKHON_SB_FPX_TP1'] }
    MAP_KASEMRAT = {0: ['SUNLAKAKHON_SB_FPX_TP1', 'MASUKGRIDLOCK_ARI_NB_FPX_TP2'],
                    1:['KASEMRAT_EB_FPX_TP2', 'KASEMRAT_NB_FPX_TP3'],
                    2:['KASEMRAT_EB_FP2_TP1', 'KASEMRAT_NB_FPX_TP3'],
                    3:['KASEMRAT_EB_FPX_TP2'] }
    MAP_ATTHAKAWI_RAMA4 = {0: ['KASEMRAT_EB_FPX_TP2'],
                           1: ['MASUKGRIDLOCK_SUKHUMVUT_FPX_TP2', 'MASUKGRIDLOCK_ARI_NB_FPX_TP2' ],
                          2:['KASEMRAT_EB_FPX_TP2_RAMA4']}
    MAP = [MAP_KLONGTEI, MAP_RAMA4, MAP_NARANONG, MAP_SUNLAKAKHON,  MAP_KASEMRAT, MAP_ATTHAKAWI_RAMA4]
    state = {}
    for i in range(len(current_phase)):
        state_attention = np.zeros(31)
        for e in MAP[i][current_phase[i]]:
            occupancy = get_occupancy_average_percent(detector[e]) 
            Index_detector = list_detector.index(e) 
            state_attention[Index_detector] = occupancy
#     print(state_attention)
        state[i] = np.concatenate((state_attention, hot_encoding_current_phase), axis=None).astype(np.float16)
    return state


In [12]:
def read_summary_xml(dateTimeObj):
    meanWaitingTime = []
    meanTravelTime = []
    meanSpeed = []
    tree = ET.parse('summary/summary3'+dateTimeObj+'.xml')
    summary = tree.getroot()
    for step in summary:
        list1 = step.attrib
        meanWaitingTime.append(float(list1["meanWaitingTime"]))
        meanTravelTime.append(float(list1["meanTravelTime"]))
        meanSpeed.append(float(list1["meanSpeed"]))
    meanWaitingTime_avg = sum(meanWaitingTime)/len(meanWaitingTime)
    meanTravelTime_avg = sum(meanTravelTime)/len(meanTravelTime)
    meanSpeed_avg = sum(meanSpeed)/len(meanSpeed)
    os.remove('summary/summary3'+dateTimeObj+'.xml')
    return meanWaitingTime_avg,meanTravelTime_avg,meanSpeed_avg

In [13]:
#reset the environment
def start(dateTimeObj):
    sumoBinary = checkBinary('sumo')
    traci.start([sumoBinary, "-c", "KASEMRAT-SUMO-UsingBookNetFile/osm.sumocfg",
                             "--summary-output", "summary/summary3"+dateTimeObj+".xml", '--start','true','--quit-on-end','true','--time-to-teleport','-1',
                '--lanechange.duration', '0.1'])

In [14]:
def get_occupancy_average_percent(detector_id): 
    #get occupancy average for all detector in list of detector_id and scale by (Vehicle Length + MinimumGap)/MinimumGap 
    #Vehicle Length = 4.62 MinimumGap = 2.37
    occupancy = (sum([traci.lanearea.getLastStepOccupancy(e)*traci.lanearea.getLength(e) for e in detector_id]))/\
                 sum([traci.lanearea.getLength(e) for e in detector_id])*((4.62+2.37)/4.62)
#     print(occupancy)
    return occupancy

In [15]:
def get_throughput():
    loopID = traci.inductionloop.getIDList()
    throughput = sum([traci.inductionloop.getLastStepVehicleNumber(i) for i in loopID if traci.inductionloop.getLastStepMeanSpeed(i) > 0])  #if traci.inductionloop.getLastStepMeanSpeed(i) > 0
    return throughput

In [16]:
def get_drawback():
#     laneID = traci.lane.getIDList()
    drawback = sum([traci.lanearea.getLastStepVehicleNumber(i) for i in NAME.loc[:,'id']])
    return drawback

In [17]:
def get_reward():
    throughput = 0
    for i in range(5):
        traci.simulationStep()
        throughput += get_throughput()
    
    drawback = get_drawback()
    reward = throughput - 0.004*drawback
    return reward, throughput, drawback

In [18]:
number_phase = [4,9,7,5,4,3]
def set_current_phase(action, current_phase):
#     if action < 4:
#         phase = action
#         current_phase[0] = phase
#     elif action < 13:
#         phase = (action-4)
#         current_phase[1] = phase
#     elif action < 20:
#         phase = (action-13)
#         current_phase[2] = phase
#     elif action < 25:
#         phase = (action-20)
#         current_phase[3] = phase
#     elif action < 29:
#         phase = (action-25)
#         current_phase[4] = phase
#     else:
#         phase = (action-29)
#         current_phase[5] = phase
#     for i in range (6):
#         traci.trafficlight.setPhase(junction_id[junction_name[i]], current_phase[i])
    
#     if current_phase[5] == 2:
#         traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT', 'id'], ['passenger'])
#         traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX', 'id'], ['passenger'])
#         traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_0_0_XSRT', 'id'], ['passenger'])
#     elif current_phase[5] == 1:
#         traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT', 'id'], ['passenger'])         
#         traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX', 'id'], ['passenger'])
#         traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_0_0_XSRT', 'id'], ['passenger'])
    
    if action[0] < 4:
        phase = action[0]
        current_phase[0] = phase
    if action[1] < 9:
        phase = action[1]
        current_phase[1] = phase
    if action[2] < 7:
        phase = action[2]
        current_phase[2] = phase
    if action[3] < 5:
        phase = action[3]
        current_phase[3] = phase
    if action[4] < 4:
        phase = action[4]
        current_phase[4] = phase
    if action[5] < 3:
        phase = action[5]
        current_phase[5] = phase
    for i in range (6):
        traci.trafficlight.setPhase(junction_id[junction_name[i]], current_phase[i])
    
    if current_phase[5] == 2:
        traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT', 'id'], ['passenger'])
        traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX', 'id'], ['passenger'])
        traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_0_0_XSRT', 'id'], ['passenger'])
    elif current_phase[5] == 1:
        traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT', 'id'], ['passenger'])         
        traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX', 'id'], ['passenger'])
        traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_0_0_XSRT', 'id'], ['passenger'])
    return 

In [19]:
''.join(str(datetime.datetime.now()).split(' '))

'2021-04-1110:48:45.756993'

In [20]:
# reward_memory = []
# def plot_durations():
#     print('show')
#     plt.figure(2)
#     plt.clf()
#     durations_t = torch.tensor(reward_memory, dtype=torch.float)
#     plt.title('Training...')
#     plt.xlabel('Step')
#     plt.ylabel('Reward')
#     plt.plot(durations_t.numpy())
#     # Take 100 episode averages and plot them too
#     if len(durations_t) >= 100:
#         means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
#         means = torch.cat((torch.zeros(99), means))
#         plt.plot(means.numpy())

#     plt.pause(0.001)  # pause a bit so that plots are updated

In [21]:
# class MyEnv3(gym.Env):
#     def __init__(self, env_config):
#         self.episode = 0
#         self.count = 0
#         self.reward = 0
#         self.rewards = {'KLONGTEI': 0,
#                   'RAMA4': 0,
#                   'NARANONG': 0,
#                   'SUNLAKAKHON':0,
#                   'KASEMRAT': 0,
#                   'ATTHAKAWI_RAMA4' : 0}
#         self.throughputs = 0
#         self.drawbacks = 0
#         self.current_phase = [1,1,1,1,1,1]
#         self.done = False
#         self.reward_memory = []
#         self.log_action = []
# #         self.action_space = gym.spaces.Discrete(32)
#         self.action_space =  {'KLONGTEI': gym.spaces.Discrete(4),
#                   'RAMA4': gym.spaces.Discrete(9),
#                   'NARANONG': gym.spaces.Discrete(7),
#                   'SUNLAKAKHON': gym.spaces.Discrete(5),
#                   'KASEMRAT': gym.spaces.Discrete(4),
#                   'ATTHAKAWI_RAMA4' : gym.spaces.Discrete(3)}
#         self.observation_space = {'KLONGTEI': gym.spaces.Box(low=-1, high=200, shape=(63,), dtype=np.float16),
#                   'RAMA4': gym.spaces.Box(low=-1, high=200, shape=(63,), dtype=np.float16),
#                   'NARANONG': gym.spaces.Box(low=-1, high=200, shape=(63,), dtype=np.float16),
#                   'SUNLAKAKHON': gym.spaces.Box(low=-1, high=200, shape=(63,), dtype=np.float16),
#                   'KASEMRAT': gym.spaces.Box(low=-1, high=200, shape=(63,), dtype=np.float16),
#                   'ATTHAKAWI_RAMA4' : gym.spaces.Box(low=-1, high=200, shape=(63,), dtype=np.float16)}
        
#         self.dateTimeObj = ''
#         print(self.dateTimeObj)
#         with open( "./Raytest/ray_results/result0000.csv" , 'w', newline='') as csv_file:
#                 header = ['rewards', 'throughput','backlog',"meanWaitingTime", "meanTravelTime","meanSpeed","action"]
#                 writer = csv.DictWriter(csv_file, fieldnames = header)
#                 writer.writeheader()
        
#     def reset(self):
        
#         self.episode += 1
#         self.log_action = []
#         self.reward_memory.append(self.rewards)
#         self.count = 0
#         dateTimeObj = datetime.datetime.now()
#         self.dateTimeObj = dateTimeObj.strftime("%d-%b-%Y-%H-%M-%S-%f")
# #         print(type(self.dateTimeObj))
#         start(self.dateTimeObj)
#         self.reward = 0
#         self.rewards = 0
#         self.throughputs = 0
#         self.backlogs = 0
#         print(self.reward_memory)
#         current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
#         state = get_state_attention(current_phase, hot_encoding_current_phase)
# #         print(state)
#         self.done = False
#         self.current_phase = current_phase
#         if len(self.reward_memory)%100 == 0:
#             print('memory',self.reward_memory[-10:])
#         return {'KLONGTEI': state,
#                   'RAMA4': state,
#                   'NARANONG': state,
#                   'SUNLAKAKHON': state,
#                   'KASEMRAT': state,
#                   'ATTHAKAWI_RAMA4' : state}
    
    
#     def step(self, action):
#         set_current_phase(action, self.current_phase)
# #         print(action)
#         current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
#         self.reward, throughput, drawback= get_reward()
#         self.rewards += self.reward
#         self.throughputs += throughput
#         self.drawbacks += drawback
#         state = get_state_attention(current_phase, hot_encoding_current_phase)
#         if np.isnan(self.reward) == True:
#             print('HELP', type(self.reward))
#         self.count += 1
#         self.current_phase = current_phase
#         self.log_action.append(str(action))
# #         print('count', self.count)
#         self.done = False
#         if self.count >= 2880: #2880
#             traci.close()
#             self.done = True
#             meanWaitingTime_avg,meanTravelTime_avg,meanSpeed_avg = read_summary_xml(self.dateTimeObj)
#             data_set = {"rewards": self.rewards, "meanWaitingTime": meanWaitingTime_avg, "meanTravelTime": meanTravelTime_avg,
#                        "meanSpeed": meanSpeed_avg, "throughput": self.throughputs, "backlog": self.backlogs/2880, 
#                         "action":self.log_action
#                        }
            
#             self.json_dump = json.dumps(data_set)
# #             if self.episode%20 == 0:
# #                 self.log_action_json = json.dumps(self.log_action)
# #                 with open( "./Raytest/ray_results/action"+self.dateTimeObj+".csv" , 'a', newline='') as csv_file:
# #                     header = ['action']
# #                     writer = csv.DictWriter(csv_file, fieldnames = header)
# #                     writer.writerow({'action': self.log_action})
                                 
#             with open( "./Raytest/ray_results/result0000.csv" , 'a', newline='') as csv_file:
#                 header = ['rewards', 'throughput','backlog',"meanWaitingTime", "meanTravelTime","meanSpeed","action"]
#                 writer = csv.DictWriter(csv_file, fieldnames = header)
#                 writer.writerow({'rewards': self.rewards, 
#                                  'throughput': self.throughputs,
#                                 'backlog': self.backlogs/2880,
#                                 "meanWaitingTime": meanWaitingTime_avg, "meanTravelTime": meanTravelTime_avg,
#                        "meanSpeed": meanSpeed_avg, "action":self.log_action})

        
#         return_state = np.array(state).astype(np.float16)
# #         print(return_state)
#         info = {"throughput": throughput,
#                 "drawback":drawback
#                 }
#         info = {**info}
# #         print(info)
#         return {'KLONGTEI': return_state,
#                   'RAMA4': return_state,
#                   'NARANONG': return_state,
#                   'SUNLAKAKHON': return_state,
#                   'KASEMRAT': return_state,
#                   'ATTHAKAWI_RAMA4' : return_state} , {'KLONGTEI': self.reward,
#                   'RAMA4': self.reward,
#                   'NARANONG': self.reward,
#                   'SUNLAKAKHON': self.reward,
#                   'KASEMRAT': self.reward,
#                   'ATTHAKAWI_RAMA4' : self.reward}, {'KLONGTEI': self.done,
#                   'RAMA4': self.done,
#                   'NARANONG': self.done,
#                   'SUNLAKAKHON': self.done,
#                   'KASEMRAT': self.done,
#                   'ATTHAKAWI_RAMA4' : self.done}, info

In [22]:
# from ray.rllib.env.multi_agent_env import MultiAgentEnv
# class IrrigationEnv(MultiAgentEnv):
#     def __init__(self, return_agent_actions = False, part=False):
#         self.num_agents = 5
#         self.observation_space = gym.spaces.Box(low=200, high=800, shape=(1,))
#         self.action_space = gym.spaces.Discrete(1)

#     def reset(self):
#         obs = {}
#         self.water = np.random.uniform(200,800)
#         for i in range(self.num_agents):
#             obs[i] = np.array([self.water])
#         return obs

#     def cal_rewards(self, action_dict):
#         self.curr_water = self.water
#         reward = 0
#         for i in range(self.num_agents):
#             water_demanded = self.water*action_dict[i][0]
#             if self.curr_water == 0:
#                 # No water is left in stream
#                 reward -= water_demanded*100 # Penalty
#             elif self.curr_water - water_demanded<0:
#                 # Water in stream is less than water demanded, withdraw all left
#                 water_needed = water_demanded - self.curr_water
#                 water_withdrawn = self.curr_water
#                 self.curr_water = 0
#                 reward += -water_withdrawn**2 + 200*water_withdrawn
#                 reward -= water_needed*100 # Penalty
#             else:
#                 # Water in stream is more than water demanded, withdraw water demanded
#                 self.curr_water -= water_demanded
#                 water_withdrawn = water_demanded
#                 reward += -water_withdrawn**2 + 200*water_withdrawn

#         return reward

#     def step(self, action_dict):
#         obs, rew, done, info = {}, {}, {}, {}

#         reward = self.cal_rewards(action_dict)

#         for i in range(self.num_agents):

#             obs[i], rew[i], done[i], info[i] = np.array([self.curr_water]), reward, True, {}

#         done["__all__"] = True
#         return obs, rew, done, info

In [23]:
from ray.rllib.env.multi_agent_env import MultiAgentEnv
class MyEnv3(MultiAgentEnv):
    def __init__(self, return_agent_actions = False, part=False):
        self.num_agents = 6
        self.observation_space = gym.spaces.Box(low=-1, high=200, shape=(63,), dtype=np.float16)
#         self.action_space = gym.spaces.Discrete(9)
#         self.action_space = gym.spaces.MultiDiscrete(np.array([4,9,7,5,4,3]))
#     gym.spaces.Box(low=np.array([0,0,0,0,0,0]), high=np.array([4,9,7,5,4,3]), dtype=np.int8)
        self.episode = 0
        self.count = 0
        self.reward = 0
        self.rewards = 0
        self.throughputs = 0
        self.drawbacks = 0
        self.current_phase = [1,1,1,1,1,1]
        self.done = False
        self.reward_memory = []
        self.log_action = []
#         self.action_space = gym.spaces.Discrete(32)
        self.dateTimeObj = ''
        print(self.dateTimeObj)
        with open( "./Raytest/ray_results/result0015.csv" , 'w', newline='') as csv_file:
                header = ['rewards', 'throughput','backlog',"meanWaitingTime", "meanTravelTime","meanSpeed","action"]
                writer = csv.DictWriter(csv_file, fieldnames = header)
                writer.writeheader()         
    def reset(self):
        self.episode += 1
        self.log_action = []
        self.reward_memory.append(self.rewards)
        self.count = 0
        dateTimeObj = datetime.datetime.now()
        self.dateTimeObj = dateTimeObj.strftime("%d-%b-%Y-%H-%M-%S-%f")
#         print(type(self.dateTimeObj))
        start(self.dateTimeObj)
        self.reward = 0
        self.rewards = 0
        self.throughputs = 0
        self.backlogs = 0
        print(self.reward_memory)
        current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
        state = get_state_attention(current_phase, hot_encoding_current_phase)
#         print(state)
        self.done = False
        self.current_phase = current_phase
        if len(self.reward_memory)%100 == 0:
            print('memory',self.reward_memory[-10:])
        return state
    def step(self, action):
        done = {}
        set_current_phase(action, self.current_phase)
#         print(action)
        current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
        self.reward, throughput, drawback= get_reward()
        self.rewards += self.reward
        self.throughputs += throughput
        self.drawbacks += drawback
        state = get_state_attention(current_phase, hot_encoding_current_phase)
        if np.isnan(self.reward) == True:
            print('HELP', type(self.reward))
        self.count += 1
        self.current_phase = current_phase
        self.log_action.append(str(action))
#         print('count', self.count)
        self.done = False
        done[0] = False
        done[1] = False
        done[2] = False
        done[3] = False
        done[4] = False
        done[5] = False
        done["__all__"] = False
        if self.count >= 20: #2880
            traci.close()
            self.done = True
            done[0] = True
            done[1] = True
            done[2] = True
            done[3] = True
            done[4] = True
            done[5] = True
            done["__all__"] = True
            meanWaitingTime_avg,meanTravelTime_avg,meanSpeed_avg = read_summary_xml(self.dateTimeObj)
            data_set = {"rewards": self.rewards, "meanWaitingTime": meanWaitingTime_avg, "meanTravelTime": meanTravelTime_avg,
                       "meanSpeed": meanSpeed_avg, "throughput": self.throughputs, "backlog": self.backlogs/2880, 
                        "action":self.log_action
                       }
            
            self.json_dump = json.dumps(data_set)
#             if self.episode%20 == 0:
#                 self.log_action_json = json.dumps(self.log_action)
#                 with open( "./Raytest/ray_results/action"+self.dateTimeObj+".csv" , 'a', newline='') as csv_file:
#                     header = ['action']
#                     writer = csv.DictWriter(csv_file, fieldnames = header)
#                     writer.writerow({'action': self.log_action})
                                 
            with open( "./Raytest/ray_results/result0015.csv" , 'a', newline='') as csv_file:
                header = ['rewards', 'throughput','backlog',"meanWaitingTime", "meanTravelTime","meanSpeed","action"]
                writer = csv.DictWriter(csv_file, fieldnames = header)
                writer.writerow({'rewards': self.rewards, 
                                 'throughput': self.throughputs,
                                'backlog': self.backlogs/2880,
                                "meanWaitingTime": meanWaitingTime_avg, "meanTravelTime": meanTravelTime_avg,
                       "meanSpeed": meanSpeed_avg, "action":self.log_action})

        
#         return_state = np.array(state).astype(np.float16)
        
#         print(return_state)
#         info = {"throughput": throughput,
#                 "drawback":drawback
#                 }
#         info = {**info}
#         print(info)
        return state , {0: self.reward,
                  1: self.reward,
                  2: self.reward,
                  3: self.reward,
                  4: self.reward,
                  5 : self.reward}, done, {}
        

#     def step(self, action_dict):
#         obs, rew, done, info = {}, {}, {}, {}

#         reward = self.cal_rewards(action_dict)

#         for i in range(self.num_agents):

#             obs[i], rew[i], done[i], info[i] = np.array([self.curr_water]), reward, True, {}

#         done["__all__"] = True
#         return obs, rew, done, info

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
import ray
from ray import tune
from ray.rllib.agents.registry import get_agent_class
from ray.rllib.models import ModelCatalog
from ray.tune import run_experiments
from ray.tune.registry import register_env
from ray.rllib.agents.ppo import PPOTrainer


# Driver code for training
def setup_and_train():

    # Create a single environment and register it
    def env_creator(_):
        return MyEnv3()
    single_env = MyEnv3()
    env_name = "MyEnv3"
    register_env(env_name, env_creator)

    # Get environment obs, action spaces and number of agents
    obs_space = single_env.observation_space
    act_space = [4,9,7,5,4,3]
    num_agents = single_env.num_agents

    # Create a policy mapping
    def gen_policy(i):
        return (None, obs_space, gym.spaces.Discrete(act_space[i]), {})

    policy_graphs = {}
    for i in range(num_agents):
        policy_graphs['agent-' + str(i)] = gen_policy(i)

    def policy_mapping_fn(agent_id):
        return 'agent-' + str(agent_id)

    # Define configuration with hyperparam and training details
    config={
                "log_level": "WARN",
                "adam_epsilon": 1e-8,
                # If not None, clip gradients during optimization at this value

                "num_gpus": 0,
                "dueling": False,
                "double_q": False,
                "num_workers": 2,
                "buffer_size": 100000,
                "framework": "tf",
                "learning_starts": 200, #2160
                "train_batch_size": 540,
        #             "num_samples": 20,
                "rollout_fragment_length": 50,
                "target_network_update_freq": 540,
                "prioritized_replay": True,
                "timesteps_per_iteration": 20, #2880
                "exploration_config": {"type": "PerWorkerEpsilonGreedy"},
                "worker_side_prioritization": True,
        #         "min_iter_time_s": 30,
                # If set, this will fix the ratio of replayed from a buffer and learned
                # on timesteps to sampled from an environment and stored in the replay
                # buffer timesteps. Otherwise, replay will proceed as fast as possible.
        #         "training_intensity": None,
                 "lr": 5e-4,
                "gamma": 0.7,
                "num_cpus_for_driver": 1,
                "num_cpus_per_worker": 1,
                "num_gpus" : 0,
#                 "model":{"fcnet_hiddens": [8, 8]},
                "multiagent": {
                    "policies": policy_graphs,
                    "policy_mapping_fn": policy_mapping_fn,
                },
                "env": MyEnv3}

    # Define experiment details
    exp_name = 'my_exp'
    exp_dict = {
            'name': exp_name,
            'run_or_experiment': 'APEX',
#             "stop": {
#                 "training_iteration": 100
#             },
             'local_dir' : "./Raytest/ray_results",
            'checkpoint_at_end' : True,
            'log_to_file' : True,
            'checkpoint_freq': 3,
            "config": config,
        }

    # Initialize ray and run
    ray.shutdown()
    ray.init()
    tune.run(**exp_dict)

if __name__=='__main__':
    setup_and_train()

2021-04-11 10:49:55,589	WARNING utils.py:479 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2021-04-11 10:49:56,457	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8267
2021-04-11 10:49:56,462	WARNING services.py:1619 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67096576 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=Xgb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 2gb.
2021-04-11 10:50:00,

Trial name,status,loc
APEX_MyEnv3_a9c0e_00000,RUNNING,


(pid=2334) WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=2334) Instructions for updating:
(pid=2334) non-resource variables are not supported in the long term
(pid=2334) WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=2334) Instructions for updating:
(pid=2334) non-resource variables are not supported in the long term
(pid=2334) 2021-04-11 10:50:05,813	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=2334) 2021-04-11 10:50:05,813	INFO trainer.py:641 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv f

(pid=2330) 
(pid=2330) 
(pid=2335) 
(pid=2335) 


(pid=2330) 2021-04-11 10:50:10,752	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=2330) 2021-04-11 10:50:10,752	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=2335) 2021-04-11 10:50:10,804	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=2335) 2021-04-11 10:50:10,804	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=2335) 2021-04-11 10:50:12,304	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=2335) 2021-04-11 10:

...

(pid=2335) Warning: Environment variable SUMO_HOME is not set, schema resolution will use slow website lookups.
(pid=2335) Warning: Environment variable SUMO_HOME is not set, schema resolution will use slow website lookups.


(pid=2335)  Retrying in 1 seconds
(pid=2335) Loading configuration ... done.
(pid=2335)  Retrying in 1 seconds
(pid=2335) Loading configuration ... done.


(pid=2330) Warning: Environment variable SUMO_HOME is not set, schema resolution will use slow website lookups.
(pid=2330) Warning: Environment variable SUMO_HOME is not set, schema resolution will use slow website lookups.


(pid=2330)  Retrying in 1 seconds
(pid=2330) Loading configuration ... done.
(pid=2330)  Retrying in 1 seconds
(pid=2330) Loading configuration ... done.
(pid=2335) [0, 39.172, 39.172, 35.012, 34.19200000000001]
(pid=2335) [0, 39.172, 39.172, 35.012, 34.19200000000001]
(pid=2330) [0, 41.09599999999999, 41.12799999999999, 51.18800000000001, 41.204]
(pid=2330) [0, 41.09599999999999, 41.12799999999999, 51.18800000000001, 41.204]
(pid=2335)  Retrying in 1 seconds
(pid=2335)  Retrying in 1 seconds
(pid=2335) Loading configuration ... done.
(pid=2335) Loading configuration ... done.


(pid=2335) Warning: Environment variable SUMO_HOME is not set, schema resolution will use slow website lookups.
(pid=2335) Warning: Environment variable SUMO_HOME is not set, schema resolution will use slow website lookups.


In [ ]:
# from typing import Tuple, Dict, List
# import gym

# from ray.rllib.utils.annotations import PublicAPI
# from ray.rllib.utils.typing import MultiAgentDict, AgentID

# # If the obs space is Dict type, look for the global state under this key.
# ENV_STATE = "state"


# def make_multi_agent(env_name_or_creator):
#     """Convenience wrapper for any sigle-agent env to be converted into MA.
#     Agent IDs are int numbers starting from 0 (first agent).
#     Args:
#         env_name_or_creator (Union[str, Callable[]]: String specifier or
#             env_maker function.
#     Returns:
#         Type[MultiAgentEnv]: New MultiAgentEnv class to be used as env.
#             The constructor takes a config dict with `num_agents` key
#             (default=1). The reset of the config dict will be passed on to the
#             underlying single-agent env's constructor.
#     Examples:
#          >>> # By gym string:
#          >>> ma_cartpole_cls = make_multi_agent("CartPole-v0")
#          >>> # Create a 2 agent multi-agent cartpole.
#          >>> ma_cartpole = ma_cartpole_cls({"num_agents": 2})
#          >>> obs = ma_cartpole.reset()
#          >>> print(obs)
#          ... {0: [...], 1: [...]}
#          >>> # By env-maker callable:
#          >>> ma_stateless_cartpole_cls = make_multi_agent(
#          ...    lambda config: StatelessCartPole(config))
#          >>> # Create a 2 agent multi-agent stateless cartpole.
#          >>> ma_stateless_cartpole = ma_stateless_cartpole_cls(
#          ...    {"num_agents": 2})
#     """

#     class MultiEnv():
#         def __init__(self, config):
#             num = config.pop("num_agents", 6)
#             if isinstance(env_name_or_creator, str):
#                 self.agents = [
#                     gym.make(env_name_or_creator) for _ in range(num)
#                 ]
#             else:
#                 self.agents = [env_name_or_creator(config) for _ in range(num)]
#             self.dones = set()
#             self.observation_space = self.agents[0].observation_space
#             self.action_space = self.agents[0].action_space

#         def reset(self):
#             self.dones = set()
#             return {i: a.reset() for i, a in enumerate(self.agents)}

#         def step(self, action_dict):
#             obs, rew, done, info = {}, {}, {}, {}
#             for i, action in action_dict.items():
#                 obs[i], rew[i], done[i], info[i] = self.agents[i].step(action)
#                 if done[i]:
#                     self.dones.add(i)
#             done["__all__"] = len(self.dones) == len(self.agents)
#             return obs, rew, done, info

#     return MultiEnv

In [ ]:
# results = tune.run(experiment_spec, 
#                                   callbacks=[MyCallback()], restore="Raytest/ray_results/experiment_apex6/APEX_3f624_00000_0_2021-04-10_15-27-23/checkpoint_81")